## Create Base Portfolios

This notebook creates Base Portfolios in Moody's Risk Modeler

In [ ]:
%load_ext autoreload
%autoreload 2

# Import required modules
import sys
import os
from pathlib import Path

# Determine the notebook's actual directory
cwd = Path.cwd()

if 'Active_' in str(cwd):
    # Working directory is set correctly, construct path to THIS notebook
    notebook_path = cwd / 'Step_02_Create_Base_Portfolios.ipynb'
else:
    # Working directory is not set correctly (e.g., /home/jovyan)
    home = Path.home()
    workspace = home / 'workspace'
    
    if workspace.exists():
        workflows = workspace / 'workflows'
        active_dirs = list(workflows.glob('Active_*'))
        
        if active_dirs:
            # Use the first Active_ directory found and point to THIS notebook
            notebook_path = active_dirs[0] / 'notebooks' / 'Stage_03_Data_Import' / 'Step_02_Create_Base_Portfolios.ipynb'
        else:
            raise RuntimeError("No Active_ cycle directory found in workspace/workflows/")
    else:
        raise RuntimeError("Workspace directory not found")

print(f"Notebook path: {notebook_path}")

# Add workspace to path
workspace_path = notebook_path.parent.parent.parent.parent
if str(workspace_path) not in sys.path:
    sys.path.insert(0, str(workspace_path))

from helpers import ux
from helpers.context import WorkContext
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.job import track_job_status
from helpers.database import execute_query
from helpers.step import Step, get_last_step_run
from helpers.irp_integration import IRPClient

In [ ]:
# Initialize work context
ux.header("Base Data Import")

# Initialize context with explicit notebook path
context = WorkContext(notebook_path=str(notebook_path))

ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")

# Initialize step execution tracking
step = Step(context)

if step.executed:
    ux.warning("⚠ This step has already been executed")
    ux.info(f"Message: {step.status_message}")
    
    last_run = get_last_step_run(step.step_id)
    if last_run:
        ux.info(f"Last run: #{last_run['run_num']}")
        ux.info(f"Status: {last_run['status']}")
        if last_run['completed_ts']:
            ux.info(f"Completed: {last_run['completed_ts'].strftime('%Y-%m-%d %H:%M:%S')}")
    
    response = ux.yes_no("Do you want to re-run this step?")
    if response:
        ux.info("Re-running step...")
        step.start(force=True)
    else:
        ux.info("Step execution skipped")
        raise SystemExit("Step already completed")

ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

### Retrieve Base Portfolio Batch

In [ ]:
# Retrieve Portfolio Creation batch from Stage_01/Step_03
ux.subheader("Retrieve Base Portfolio Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if 'Portfolio Creation' not in batches:
    raise ValueError(f"Portfolio Creation batch not found. Available: {list(batches.keys())}")

portfolio_batch_id = int(batches['Portfolio Creation'])

ux.success(f"✓ Retrieved Portfolio Creation batch: ID={portfolio_batch_id}")
step.log(f"Retrieved Portfolio Creation batch: ID={portfolio_batch_id}")

### Review Base Portfolio Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(portfolio_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(portfolio_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Show sample configurations
if job_count > 0:
    ux.info("\nSample EDM Configurations (first 5):")
    for i, job in enumerate(jobs[:5]):
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            ux.info(f"  {i+1}. {config.get('Portfolio', 'N/A'), config.get('Database', 'N/A')}")

step.log(f"Verified batch: {job_count} jobs ready for submission")

### Submit Base Portfolio Batch to Moody's

In [ ]:
# Submit batch to Moody's API
ux.subheader("Submit Batch to Moody's")

ux.info("")
ux.info("Submission Process:")
ux.info("  • Each job will create a Portfolio in Moody's")
ux.info("  • Jobs will transition to SUBMITTED status")
ux.info("  • Batch will transition to ACTIVE status")
ux.info("")

# Confirm submission
proceed = ux.yes_no("Submit batch to Moody's API?")

if not proceed:
    step.log("User cancelled batch submission")
    raise SystemExit("User cancelled batch submission")

# Submit
ux.info("\nSubmitting batch...")

result = submit_batch(portfolio_batch_id, IRPClient())

# Display results
ux.success(f"\n✓ Batch submission completed")
ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
ux.info(f"  Status: {result['batch_status']}")

# Check for errors
failed_count = len([j for j in result['jobs'] if 'error' in j])
if failed_count > 0:
    ux.warning(f"\n⚠ {failed_count} job(s) failed to submit")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

### Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

try:
    # Prepare output data
    output_data = {
        'batch_id': portfolio_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    
    # Complete the step
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("✓ BASE PORTFOLIO BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs']} job(s) to Moody's API")
    if failed_count > 0:
        ux.warning(f"⚠ {failed_count} job(s) failed to submit")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNote: Portfolio Creation is synchronous - all jobs complete immediately")
    ux.info("Next: Proceed to MRI Import or other data loading steps")

except Exception as e:
    ux.error(f"✗ Step completion failed: {str(e)}")
    step.fail(str(e))
    raise